In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler
from tqdm.auto import tqdm
import evaluate
import uuid

from classification import ReviewDataSet, load_and_prepare_data, train_model, save_model, eval


In [3]:
# load data
print("loading data...\n")
path = "drive/MyDrive/yelp_reviews/"
df = load_and_prepare_data(path+"yelp_academic_dataset_review.json", sample_size=1000, COLAB=True)

loading data...

Mounted at /content/drive


In [4]:
# create dataset
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased', use_fast=False)
review_dataset = ReviewDataSet(tokenizer, df)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [6]:
# train and val split
train_size = int(0.8 * len(review_dataset))
val_size = len(review_dataset) - train_size
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(review_dataset, [train_size, val_size], generator)
print(f'train and validation dataset sizes: {len(train_dataset), len(val_dataset)}\n')

train and validation dataset sizes: (296, 74)



In [7]:
# dataloader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [8]:
# initialize model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=5)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_params = sum([p.numel() for p in model.parameters()])
print(f"model params: {num_params}\n")

epochs = 3
train_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=train_steps)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model params: 108314117



In [ ]:
# set device and send model to device
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
model.to(device)

In [10]:
device

device(type='cuda')

In [11]:
# train model
print("beginning model training...\n")
train_model(model, epochs, train_dataloader, val_dataloader, train_steps, optimizer, lr_scheduler, path)
print("complete")

beginning model training...



  0%|          | 0/57 [00:00<?, ?it/s]

model id :: 6f2814fd-7580-481f-8c6a-f5ff98245250
evaluating model...



Accuracy: 0.28378378378378377
saving model at epoch: 1, step: 9
current best accuracy: 0.28378378378378377

evaluating model...

Accuracy: 0.43243243243243246
saving model at epoch: 1, step: 18
current best accuracy: 0.43243243243243246

evaluating model...

Accuracy: 0.5540540540540541
saving model at epoch: 2, step: 9
current best accuracy: 0.5540540540540541

evaluating model...

Accuracy: 0.581081081081081
saving model at epoch: 2, step: 18
current best accuracy: 0.581081081081081

evaluating model...

Accuracy: 0.5945945945945946
saving model at epoch: 3, step: 9
current best accuracy: 0.5945945945945946

evaluating model...

Accuracy: 0.5675675675675675
skipping model save...
current best accuracy: 0.5945945945945946

complete


In [ ]:
# load pretrained model
# path = "drive/MyDrive/yelp_reviews/bert_model"
path = "drive/MyDrive/yelp_reviews/outputs"

model = AutoModelForSequenceClassification.from_pretrained(path)
model.to(device)